<a href="https://colab.research.google.com/github/sofaemha/archive/blob/main/2025-10/spk-ahp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [64]:
from rich import print
from rich.panel import Panel
from rich.padding import Padding
from rich.text import Text
from rich.table import Table
from rich.prompt import Prompt
import numpy as np

## System Function

In [35]:
def panel(text, title="", subtitle="", justify="center"):
  print(Panel(Text(text, justify=justify), title=title, subtitle=subtitle))

In [46]:
def space(size):
  print(Padding("", size))

In [166]:
def transpose(array):
    matrix = array.reshape(-1, 1)
    row, column = matrix.shape
    return matrix, row, column

In [197]:
def table_custom(scol, srow, array, title="", tcol="", trow="", icol=True, irow=True, index=1):
  table = Table(show_header=True)
  table.add_column(title, justify="center", no_wrap=True)

  for i in range(scol):
    col_label = f"{tcol}{i+index}" if icol else f"{tcol}"
    table.add_column(col_label, justify="center")

  for i in range(srow):
    row_label = f"{trow}{i+index}" if irow else f"{trow}"
    row_data = [f"{array[i][j]:.7f}".rstrip('0').rstrip('.') for j in range(scol)]
    table.add_row(row_label, *row_data)

  print(table)

In [126]:
def table(size, array, title="", tcol="", trow="", index=True):
  table = Table(show_header=True)
  table.add_column(title, justify="center", no_wrap=True)

  for i in range(size):
    col_label = f"{tcol}{i+1}" if index else f"{tcol}"
    table.add_column(col_label, justify="center")

  for i in range(size):
    row_label = f"{trow}{i+1}" if index else f"{trow}"
    row_data = [f"{array[i][j]:.7f}".rstrip('0').rstrip('.') for j in range(size)]
    table.add_row(row_label, *row_data)

  print(table)

---

## Main Function

In [162]:
def ss1(n, normalized):
    print("\nNormalized Pairwise Matrix:\n")
    table(n, normalized, "Criteria", "C", "C")


In [179]:
def ss2(sum_of_normalized):
    print("\nSum of Normalized Matrix:\n")
    matrix, row, column = transpose(sum_of_normalized)
    table_custom(column, row, matrix, "Criteria", "Value", "C", False)

In [180]:
def ss3(weights):
    print("\nAverage of Normalized Matrix (Priority Vector / Criteria Weights):\n")
    matrix, row, column = transpose(weights)
    table_custom(column, row, matrix, "Criteria", "Priority", "C", False)

In [181]:
def ss4(product):
    print("\nCalculate Matrix Product:\n")
    matrix, row, column = transpose(product)
    table_custom(column, row, matrix, "Criteria", "Product", "C", False)

In [182]:
def ss5(cm):
    print("\nCalculate Consistency Measure (CM):\n")
    matrix, row, column = transpose(cm)
    table_custom(column, row, matrix, "Criteria", "CM", "C", False)

In [199]:
def ss6(ri_values):
    print("\nOrdo Matrix Table of Ratio Index (RI):\n")
    ri_array = np.array(list(ri_values.values()))
    matrix, row, column = transpose(ri_array)
    table_custom(column, row, matrix, "Ordo", "Ratio", icol=False, index=0)

---

In [12]:
def normalize_matrix(matrix):
    """Normalize the matrix by dividing each element by the column sum."""
    col_sums = np.sum(matrix, axis=0)
    return matrix / col_sums

In [13]:
def calculate_priority_vector(normalized_matrix):
    # Calculate the priority vector by averaging the rows.
    return np.mean(normalized_matrix, axis=1)

In [175]:
def calculate_lambda_max(matrix, priority_vector):
    # Calculate the maximum eigenvalue (lambda_max).
    product = np.dot(matrix, priority_vector)
    ss4(product)
    space(1)

    cm = product / priority_vector
    ss5(cm)
    space(1)

    return np.mean(cm)

In [15]:
def calculate_consistency_index(lambda_max, n):
    # Calculate the Consistency Index (CI).
    return (lambda_max - n) / (n - 1)

In [194]:
def get_random_index(n):
    # Get the Random Index (RI) for a given matrix size n.
    ri_values = {
        1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12,
        6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49
    }

    ss6(ri_values)
    space(1)

    return ri_values.get(n, 1.49)  # Default to 1.49 for n > 10

In [17]:
def calculate_consistency_ratio(ci, ri):
    # Calculate the Consistency Ratio (CR).
    return ci / ri

---

In [188]:
def ahp(pairwise_matrix):
    # Perform AHP calculation on a pairwise comparison matrix.
    n = pairwise_matrix.shape[0]

    # Normalize the matrix
    normalized = normalize_matrix(pairwise_matrix)
    ss1(n, normalized)
    space(1)

    sum_of_normalized = np.sum(normalized, axis=1)
    ss2(sum_of_normalized)
    space(1)

    # Calculate priority vector (weights)
    weights = calculate_priority_vector(normalized)
    ss3(weights)
    space(1)

    # Calculate lambda_max
    lambda_max = calculate_lambda_max(pairwise_matrix, weights)
    print("\nAverage of Consistency Measure (Lambda Max):\n")
    print(lambda_max)
    space(1)

    # Calculate CI
    ci = calculate_consistency_index(lambda_max, n)
    print("\nCalculate Consistency Index (CI):\n")
    print(ci)
    space(1)

    # Get RI
    ri = get_random_index(n)
    print("\nCalculate Ratio Index (RI):\n")
    print(ri)
    space(1)

    # Calculate CR
    cr = calculate_consistency_ratio(ci, ri)
    print("\nCalculate Consistency Ratio (CR):\n")
    print(cr)
    space(1)

    return weights, cr

---

## Additional Function

In [68]:
def s1(number_criteria, criteria_matrix):
  # Fill the matrix
  print("\nEnter the pairwise comparison values for criteria (use Saaty scale: 1-9, and reciprocals)")
  space(1)

  for i in range(number_criteria):
      for j in range(i + 1, number_criteria):
          value = float(Prompt.ask(f"\nCompare criterion {i+1} to {j+1}", choices=["1", "2", "3", "4", "5", "6", "7", "8", "9"]))
          criteria_matrix[i, j] = value
          criteria_matrix[j, i] = 1 / value
      criteria_matrix[i, i] = 1.0  # Diagonal is 1


In [184]:
def s2(number_criteria, criteria_matrix):
  print("\nCriteria Pairwise Matrix:\n")
  table(number_criteria, criteria_matrix, "Criteria", "C", "C")

In [209]:
def s3(criteria_cr):
  if criteria_cr > 0.1:
      print("Warning: CR > 0.1, the matrix may be inconsistent.")
  else:
      print("Info: CR < 0.1, the matrix is consistent.")

In [212]:
def s4(c, number_alternatives, alt_matrix):
  print(f"\nPairwise comparisons for alternatives under criterion {c+1}\n")
  for i in range(number_alternatives):
      for j in range(i + 1, number_alternatives):
          value = float(Prompt.ask(f"\nCompare alternative {i+1} to {j+1} under criterion {c+1}", choices=["1", "2", "3", "4", "5", "6", "7", "8", "9"]))
          alt_matrix[i, j] = value
          alt_matrix[j, i] = 1 / value
      alt_matrix[i, i] = 1.0


---

In [216]:
def main():
  panel("\nAnalytical Hierarchy Process\nCalculator\n")
  space(1)

  # Input number of criteria
  number_criteria = int(Prompt.ask("Enter the number of criteria"))
  space(1)

  # Initialize pairwise comparison matrix for criteria
  criteria_matrix = np.zeros((number_criteria, number_criteria))

  s1(number_criteria, criteria_matrix)
  space(1)

  panel("\nPairwise Matrix\n")
  space(1)

  s2(number_criteria, criteria_matrix)
  space(1)

  # Calculate criteria weights
  criteria_weights, criteria_cr = ahp(criteria_matrix)

  print(f"\nConsistency Ratio (CR): {criteria_cr:.4f}\n")
  s3(criteria_cr)
  space(1)
  print("\n====================================================================================================")

  # Input number of alternatives
  number_alternatives = int(Prompt.ask("Enter the number of alternatives"))
  space(1)

  # For each criterion, create a pairwise matrix for alternatives
  alt_matrix = np.zeros((number_alternatives, number_alternatives))
  alternative_matrices = []
  alternative_weights = []

  panel("\nPairwise Matrix\n")
  space(1)

  for c in range(number_criteria):
      s4(c, number_alternatives, alt_matrix)

      print(f"\nAlternative Matrix for Criterion {c+1}:")
      print(alt_matrix)

      alt_weights, alt_cr = ahp(alt_matrix)
      alternative_matrices.append(alt_matrix)
      alternative_weights.append(alt_weights)

      print(f"\nAlternative Weights for Criterion {c+1}:")
      for i, w in enumerate(alt_weights, 1):
          print(f"Alternative {i}: {w:.4f}")
      print(f"\nConsistency Ratio (CR): {alt_cr:.4f}")
      if alt_cr > 0.1:
          print("Warning: CR > 0.1, the matrix may be inconsistent.")
      else:
          print("Info: CR < 0.1, the matrix is consistent.")


In [52]:
def fn():

    # Calculate overall scores
    overall_scores = np.zeros(num_alternatives)
    for a in range(num_alternatives):
        for c in range(num_criteria):
            overall_scores[a] += criteria_weights[c] * alternative_weights[c][a]

    print("\nOverall Scores:")
    for i, score in enumerate(overall_scores, 1):
        print(f"Alternative {i}: {score:.4f}")

    # Rank the alternatives
    ranked = np.argsort(overall_scores)[::-1]
    print("\nRanking:")
    for rank, idx in enumerate(ranked, 1):
        print(f"Rank {rank}: Alternative {idx+1} (Score: {overall_scores[idx]:.4f})")

## Execute

In [ ]:
if __name__ == "__main__":
    main()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                          Analytical Hierarchy Process                                           │
│                                                   Calculator                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Enter the number of criteria:

4


Enter the pairwise comparison values for criteria (use Saaty scale: 1-9, and reciprocals)

Compare criterion 1 to 2 [1/2/3/4/5/6/7/8/9]:

...

Alternative 3: 0.1638

Consistency Ratio (CR): 0.0079

Info: CR < 0.1, the matrix is consistent.

Pairwise comparisons for alternatives under criterion 2

Compare alternative 1 to 2 under criterion 2 [1/2/3/4/5/6/7/8/9]: